In [1]:
%matplotlib widget

In [2]:
from pathlib import Path
from split_dataset import SplitDataset
from skimage import io
import numpy as np
from scipy.stats import zscore
from bouter.utilities import fast_pearson
import matplotlib.pyplot as plt
from matplotlib.colors import hex2color
from lotr.plotting import add_anatomy_scalebar, add_cbar

In [3]:
path = Path(r"C:\Users\You\Documents\test")

aligned_ds = SplitDataset(path/"rotated")
anatomy_ds = SplitDataset(path/"anatomy")
mask_all = io.imread(path/"mask_rotated.tif", plugin="tifffile")

In [4]:
plane = 6
bin_px = 10
crop = slice(20,-20)

aligned = aligned_ds[:,plane,crop,crop]
anatomy = anatomy_ds[plane,crop,crop]
mask = mask_all[plane,crop,crop]

px_masked = np.argwhere(mask>=1)
px_str = [str(x0)+str(y0) for x0, y0 in px_masked]
x_loc = np.unique(px_masked[:,0])
y_loc = np.unique(px_masked[:,1])
x_bins = np.digitize(x_loc, np.arange(x_loc.min(), x_loc.max()+1, bin_px)) - 1
y_bins = np.digitize(y_loc, np.arange(y_loc.min(), y_loc.max()+1, bin_px)) - 1
n_bin_x = x_bins.max() + 1
n_bin_y = y_bins.max() + 1

traces = np.full((aligned.shape[0], n_bin_x*n_bin_y), np.nan)
roi_img = np.full(anatomy.shape, np.nan)
for x in range(n_bin_x):
    for y in range(n_bin_y):
        px_here = np.array([(x0, y0) for x0 in x_loc[x_bins==x] for y0 in y_loc[y_bins==y]])
        px_here_str = [str(x0)+str(y0) for x0, y0 in px_here]
        px_here = px_here[np.isin(px_here_str, px_str)]
        if len(px_here) > 0:
            traces[:, x*n_bin_y+y] = np.nanmean(aligned[:, px_here[:,0], px_here[:,1]], axis=-1)
            roi_img[px_here[:,0], px_here[:,1]] = x*n_bin_y+y
traces = zscore(traces, axis=0, nan_policy='omit')

In [5]:
px_size = 0.35
selected_bins = [105, 135, 175]
vmax = np.percentile(anatomy, 99)
extent = (0, px_size*anatomy.shape[-1],)*2

fig, ax = plt.subplots(3,2, figsize=(6,9))

for i, this_bin in enumerate(selected_bins):
    this = traces[:,this_bin]
    corr_img = roi_img.copy()
    for b in range(traces.shape[1]):
        that = traces[:,b]
        if not np.isnan(that).all():
            corr_img[corr_img==b] = fast_pearson(this, that)
    bin_img = np.zeros(roi_img.shape + (4,))
    bin_img[roi_img==this_bin, :] = hex2color("#1b9e77") + (1,)
    ax[i,0].imshow(anatomy.T, cmap="gray_r", vmax=vmax, extent=extent, origin="upper")
    ax[i,0].imshow(bin_img.transpose(1,0,2), extent=extent, origin="upper")
    ax[i,1].imshow(anatomy.T, cmap="gray_r", vmax=vmax, extent=extent, origin="upper")
    im = ax[i,1].imshow(corr_img.T, cmap="RdBu_r", vmax=1, vmin=-1, extent=extent, origin="upper")

for a in ax.flat:
    a.axis("off")
    
add_anatomy_scalebar(ax[i,0], pos=[0,0], length=40, fontsize=8)
add_cbar(im, ax[i,1], inset_loc=(1.1, 0.05, 0.05, 0.15), ticks=(-1,1))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
plt.savefig(path/"Fig5c.pdf")